In [25]:
import os
import time
import datetime
import tensorflow as tf
import pandas as pd
import numpy as np
import re
import itertools
from collections import Counter
import data_helpers
from text_cnn import TextCNN
from tensorflow.contrib import learn

In [27]:

# Parameters
# ==================================================

# Data loading params
tf.flags.DEFINE_float("dev_sample_percentage", .1, "Percentage of the training data to use for validation")
tf.flags.DEFINE_string("positive_data_file", "./data/rt-polaritydata/rt-polarity.pos", "Data source for the positive data.")
tf.flags.DEFINE_string("negative_data_file", "./data/rt-polaritydata/rt-polarity.neg", "Data source for the negative data.")
tf.flags.DEFINE_string("tweets","./data/text_emotion.csv", "Data source from crowdflower")
tf.flags.DEFINE_string("isear","./data/isear.csv", "Data source from ISEAR")

# Model Hyperparameters
tf.flags.DEFINE_integer("embedding_dim", 128, "Dimensionality of character embedding (default: 128)")
tf.flags.DEFINE_string("filter_sizes", "3,4,5", "Comma-separated filter sizes (default: '3,4,5')")
tf.flags.DEFINE_integer("num_filters", 128, "Number of filters per filter size (default: 128)")
tf.flags.DEFINE_float("dropout_keep_prob", 0.5, "Dropout keep probability (default: 0.5)")
tf.flags.DEFINE_float("l2_reg_lambda", 0.0, "L2 regularization lambda (default: 0.0)")

# Training parameters
tf.flags.DEFINE_integer("batch_size", 64, "Batch Size (default: 64)")
tf.flags.DEFINE_integer("num_epochs", 200, "Number of training epochs (default: 200)")
tf.flags.DEFINE_integer("evaluate_every", 100, "Evaluate model on dev set after this many steps (default: 100)")
tf.flags.DEFINE_integer("checkpoint_every", 100, "Save model after this many steps (default: 100)")
tf.flags.DEFINE_integer("num_checkpoints", 5, "Number of checkpoints to store (default: 5)")
# Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")

FLAGS = tf.flags.FLAGS
FLAGS._parse_flags()
print("\nParameters:")
for attr, value in sorted(FLAGS.__flags.items()):
    print("{}={}".format(attr.upper(), value))
print("")

ArgumentError: argument --dev_sample_percentage: conflicting option string(s): --dev_sample_percentage

In [3]:
def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()



In [4]:
data = pd.read_csv(FLAGS.tweets)

In [5]:
data.head()

,tweet_id,sentiment,author,content
0,1956967341,neutral,xoshayzers,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...
3,1956967789,happy,czareaquino,wants to hang out with friends SOON!
4,1956968416,neutral,xkilljoyx,@dannycastillo We want to trade with someone w...


In [6]:
x_text = data["content"]

In [7]:
data["sentiment"].unique()

array(['neutral', 'sadness', 'happy', 'worry', 'surprise', 'love', 'fun',
       'hate', 'boredom', 'relief', 'anger'], dtype=object)

In [8]:
    data['sentiment'] = data['sentiment'].map({'sadness': [1,0,0,0,0,0,0,0,0,0,0], 'happy': [0,1,0,0,0,0,0,0,0,0,0], 'neutral': [0,0,1,0,0,0,0,0,0,0,0], 'worry': [0,0,0,1,0,0,0,0,0,0,0], 'surprise': [0,0,0,0,1,0,0,0,0,0,0], 'love': [0,0,0,0,0,1,0,0,0,0,0], 'fun': [0,0,0,0,0,0,1,0,0,0,0], 'hate': [0,0,0,0,0,0,0,1,0,0,0], 'boredom': [0,0,0,0,0,0,0,0,1,0,0], 'relief': [0,0,0,0,0,0,0,0,0,1,0], 'anger': [0,0,0,0,0,0,0,0,0,0,1]})

In [9]:
y = data['sentiment']

In [10]:
x_text.count()

40000

In [11]:
y.count()

40000

In [12]:
x_text[0]

'@tiffanylue i know  i was listenin to bad habit earlier and i started freakin at his part =['

In [13]:
x_text=[data_helpers.clean_str(sent) for sent in x_text]

In [14]:
x_text[0]

'tiffanylue i know i was listenin to bad habit earlier and i started freakin at his part'

In [15]:
y[0]

[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]

In [16]:
# Build vocabulary
max_document_length = max([len(x.split(" ")) for x in x_text])
vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length)
x = np.array(list(vocab_processor.fit_transform(x_text) ))
y = np.array(list(y))

In [17]:
x

array([[    1,     2,     3, ...,     0,     0,     0],
       [   16,    17,    18, ...,     0,     0,     0],
       [   27,    28,    29, ...,     0,     0,     0],
       ..., 
       [  481,  3360,    73, ...,     0,     0,     0],
       [ 4790, 14844,  2465, ...,     0,     0,     0],
       [47325, 25561,  3543, ...,     0,     0,     0]])

In [18]:
y

array([[0, 0, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [19]:
# Randomly shuffle data
np.random.seed(10)
shuffle_indices = np.random.permutation(np.arange(len(y)))
x_shuffled = x[shuffle_indices]
y_shuffled = y[shuffle_indices]

In [20]:
x_shuffled

array([[35661,  3625,   310, ...,     0,     0,     0],
       [    2,    91,  6532, ...,     0,     0,     0],
       [23080,     2,   304, ...,     0,     0,     0],
       ..., 
       [36827,  9308,   354, ...,     0,     0,     0],
       [  303,  5114,     2, ...,     0,     0,     0],
       [   59,   638,   907, ...,     0,     0,     0]])

In [21]:
# Split train/test set
# TODO: This is very crude, should use cross-validation
dev_sample_index = -1 * int(FLAGS.dev_sample_percentage * float(len(y)))
x_train, x_dev = x_shuffled[:dev_sample_index], x_shuffled[dev_sample_index:]
y_train, y_dev = y_shuffled[:dev_sample_index], y_shuffled[dev_sample_index:]
print x_train.shape[1]

print("Vocabulary Size: {:d}".format(len(vocab_processor.vocabulary_)))
print("Train/Dev split: {:d}/{:d}".format(len(y_train), len(y_dev)))

103
Vocabulary Size: 48151
Train/Dev split: 36000/4000


In [22]:
y_train.shape[1]

11

In [23]:
#data helper function
def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]


In [24]:
# Training
# ==================================================

with tf.Graph().as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=FLAGS.allow_soft_placement,
      log_device_placement=FLAGS.log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        cnn = TextCNN(
            sequence_length=x_train.shape[1],
            num_classes=y_train.shape[1],
            vocab_size=len(vocab_processor.vocabulary_),
            embedding_size=FLAGS.embedding_dim,
            filter_sizes=list(map(int, FLAGS.filter_sizes.split(","))),
            num_filters=FLAGS.num_filters,
            l2_reg_lambda=FLAGS.l2_reg_lambda)

        # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer(1e-3)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

        # Keep track of gradient values and sparsity (optional)
        grad_summaries = []
        for g, v in grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.summary.histogram("grad/hist".format(v.name), g)
                sparsity_summary = tf.summary.scalar("grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
        grad_summaries_merged = tf.summary.merge(grad_summaries)

        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", cnn.loss)
        acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)

        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=FLAGS.num_checkpoints)

        # Write vocabulary
        vocab_processor.save(os.path.join(out_dir, "vocab"))

        # Initialize all variables
        sess.run(tf.global_variables_initializer())

        def train_step(x_batch, y_batch):
            """
            A single training step
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: FLAGS.dropout_keep_prob
            }
            _, step, summaries, loss, accuracy = sess.run(
                [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            train_summary_writer.add_summary(summaries, step)

        def dev_step(x_batch, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: 1.0
            }
            step, summaries, loss, accuracy = sess.run(
                [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            if writer:
                writer.add_summary(summaries, step)

        # Generate batches
        batches = data_helpers.batch_iter(
            list(zip(x_train, y_train)), FLAGS.batch_size, FLAGS.num_epochs)
        # Training loop. For each batch...
        for batch in batches:
            x_batch, y_batch = zip(*batch)
            train_step(x_batch, y_batch)
            current_step = tf.train.global_step(sess, global_step)
            if current_step % FLAGS.evaluate_every == 0:
                print("\nEvaluation:")
                dev_step(x_dev, y_dev, writer=dev_summary_writer)
                print("")
            if current_step % FLAGS.checkpoint_every == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print("Saved model checkpoint to {}\n".format(path))


Writing to /home/ubuntu/cnn-text-classification-tf/runs/1491227132

2017-04-03T19:15:33.646996: step 1, loss 6.50659, acc 0.0625
2017-04-03T19:15:33.948640: step 2, loss 5.35337, acc 0.078125
2017-04-03T19:15:34.226971: step 3, loss 4.78166, acc 0.078125
2017-04-03T19:15:34.502992: step 4, loss 4.58668, acc 0.15625
2017-04-03T19:15:34.791156: step 5, loss 5.01657, acc 0.15625
2017-04-03T19:15:35.115203: step 6, loss 4.60659, acc 0.171875
2017-04-03T19:15:35.430824: step 7, loss 3.47901, acc 0.25
2017-04-03T19:15:35.722640: step 8, loss 4.30357, acc 0.25
2017-04-03T19:15:36.012136: step 9, loss 4.97262, acc 0.171875
2017-04-03T19:15:36.323223: step 10, loss 4.71274, acc 0.203125
2017-04-03T19:15:36.645066: step 11, loss 4.25187, acc 0.171875
2017-04-03T19:15:36.995913: step 12, loss 4.22511, acc 0.1875
2017-04-03T19:15:37.303179: step 13, loss 4.64774, acc 0.1875
2017-04-03T19:15:37.658844: step 14, loss 4.71993, acc 0.15625
2017-04-03T19:15:37.947915: step 15, loss 4.07658, acc 0.14062

KeyboardInterrupt: 